In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import joblib
from matplotlib import rcParams

# 设置中文字体（这里以SimHei字体为例）
rcParams['font.sans-serif'] = ['SimHei']  # 指定中文字体
rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 1. 数据加载与预处理
def load_mat_files_from_directory(directory_path):
    """
    从指定目录加载所有.mat文件，并合并load_data和strain_data。
    
    参数:
        directory_path (str): .mat文件所在的目录路径。
        
    返回:
        load_list (list): 所有.mat文件中的load_data列表。
        strain_list (list): 所有.mat文件中的strain_data列表。
    """
    load_list = []
    strain_list = []
    
    # 遍历目录中的所有文件
    for filename in os.listdir(directory_path):
        if filename.endswith('.mat'):
            file_path = os.path.join(directory_path, filename)
            mat = scipy.io.loadmat(file_path)
            
            # 提取数据
            current_load = mat.get('load_data')
            current_strain = mat.get('strain_data')
            
            if current_load is None or current_strain is None:
                print(f"文件 {filename} 缺少 'load_data' 或 'strain_data'，已跳过。")
                continue
            
            load_list.append(current_load)
            strain_list.append(current_strain)
    
    if not load_list or not strain_list:
        raise ValueError("未找到任何包含 'load_data' 和 'strain_data' 的.mat文件。")
    
    # 合并所有加载的数据
    combined_load = np.concatenate(load_list, axis=1)  # 假设第二维是样本数
    combined_strain = np.concatenate(strain_list, axis=1)
    
    return combined_load, combined_strain

# 指定包含多个.mat文件的文件夹路径
mat_directory = r'../../shared-nvme/PythonG4090/data'  # 修改为您的文件夹路径

# 从文件夹中加载所有.mat文件的数据
combined_load_data, combined_strain_data = load_mat_files_from_directory(mat_directory)

# 转置数据，使时间为第一个维度
strain = combined_strain_data.T  # 形状: (总样本数, 10)
load = combined_load_data.T      # 形状: (总样本数, 1)

# 确保 load 是二维的，形状为 (样本数, 1)
if load.ndim > 2:
    load = load.squeeze()
if load.ndim == 1:
    load = load.reshape(-1, 1)
elif load.shape[1] != 1:
    load = load[:, 0:1]

# 标准化数据
strain_scaler = StandardScaler()
load_scaler = StandardScaler()

strain = strain_scaler.fit_transform(strain)  # 形状: (总样本数, 10)
load = load_scaler.fit_transform(load)        # 形状: (总样本数, 1)

# 保存scalers，以便后续使用
joblib.dump(strain_scaler, 'strain_scaler.save')
joblib.dump(load_scaler, 'load_scaler.save')

# 2. 数据切片
def create_time_windows(data, window_size=2048, overlap=0.5):
    """
    将数据切分成时间窗口。
    
    参数:
        data (numpy.ndarray): 输入数据，形状为 (样本数, 特征数)。
        window_size (int): 每个窗口的大小。
        overlap (float): 窗口之间的重叠比例。
        
    返回:
        windows (numpy.ndarray): 切分后的数据，形状为 (窗口数, window_size, 特征数)。
    """
    step = int(window_size * (1 - overlap))
    num_samples = data.shape[0]
    num_features = data.shape[1]
    
    num_windows = (num_samples - window_size) // step + 1
    windows = np.zeros((num_windows, window_size, num_features))
    
    for i in range(num_windows):
        start = i * step
        end = start + window_size
        windows[i] = data[start:end]
    
    return windows

# 定义窗口大小和重叠比例
window_size = 2048
overlap = 0.5

# 切片应变数据和载荷数据
strain_windows = create_time_windows(strain, window_size=window_size, overlap=overlap)  # 形状: (num_windows, 2048, 10)
load_windows = create_time_windows(load, window_size=window_size, overlap=overlap)        # 形状: (num_windows, 2048, 1)

print(f'应变窗口形状: {strain_windows.shape}')  # 例如: (num_windows, 2048, 10)
print(f'载荷窗口形状: {load_windows.shape}')    # 例如: (num_windows, 2048, 1)

# 3. 创建数据集
class StrainLoadTimeDomainDataset(Dataset):
    def __init__(self, strain_windows, load_windows):
        self.strain = torch.tensor(strain_windows, dtype=torch.float32)  # 形状: (num_windows, 2048, 10)
        self.load = torch.tensor(load_windows, dtype=torch.float32)      # 形状: (num_windows, 2048, 1)

    def __len__(self):
        return self.strain.shape[0]

    def __getitem__(self, idx):
        # 返回应变序列和载荷序列
        strain_sample = self.strain[idx]  # 形状: (2048, 10)
        load_sample = self.load[idx]      # 形状: (2048, 1)
        return strain_sample, load_sample

# 拆分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(
    strain_windows, load_windows, test_size=0.2, random_state=42
)

# 创建数据集对象
train_dataset = StrainLoadTimeDomainDataset(X_train, y_train)
val_dataset = StrainLoadTimeDomainDataset(X_val, y_val)

# 创建数据加载器
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

应变窗口形状: (1278, 2048, 10)
载荷窗口形状: (1278, 2048, 1)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/base/mambaforge/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/base/mambaforge/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/base/mambaforge/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/base/mambaforge/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/base/mambaforge/lib/pyth

In [2]:
# 4. 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        """
        初始化LSTM模型。
        
        参数:
        - input_size: 输入特征的数量（应变数据的特征数，例如10）
        - hidden_size: LSTM隐藏层大小
        - num_layers: LSTM层数
        - output_size: 输出特征的数量（载荷数据的特征数，例如1）
        - dropout: dropout概率
        """
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # 定义全连接层
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # 初始化隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # (num_layers, batch_size, hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # 前向传播LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_length, hidden_size)
        
        # 通过全连接层
        out = self.fc(out)  # (batch_size, seq_length, output_size)
        return out

# 初始化模型参数
input_size = 10        # 应变数据的特征数量
hidden_size = 1024     # LSTM隐藏层大小，可以根据需要调整
num_layers = 4        # LSTM层数，可以根据需要调整
output_size = 1        # 载荷数据的特征数量

# 实例化模型
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size)

# 移动模型到GPU（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 5. 训练与验证流程
num_epochs = 300  # 根据需要调整
best_val_loss = float('inf')
patience = 100  # 早停耐心值
counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for batch_idx, (strain, load) in enumerate(train_loader):
        strain = strain.to(device)  # 形状: (batch_size, 2048, 10)
        load = load.to(device)      # 形状: (batch_size, 2048, 1)
        
        # 前向传播
        outputs = model(strain)     # 形状: (batch_size, 2048, 1)
        loss = criterion(outputs, load)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * strain.size(0)
    
    avg_train_loss = train_loss / len(train_loader.dataset)
    
    # 验证
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for strain, load in val_loader:
            strain = strain.to(device)
            load = load.to(device)
            
            outputs = model(strain)
            loss = criterion(outputs, load)
            val_loss += loss.item() * strain.size(0)
    
    avg_val_loss = val_loss / len(val_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}')
    
    # 早停检查
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        # 保存最好的模型
        torch.save(model.state_dict(), 'best_lstm_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print("早停触发！")
            break

print("训练完成！")

# 6. 载入最好的模型（如果需要）
model.load_state_dict(torch.load('best_lstm_model.pth'))

# 7. 评估模型
model.eval()
test_losses = []
with torch.no_grad():
    for strain, load in val_loader:
        strain = strain.to(device)
        load = load.to(device)
        
        outputs = model(strain)
        loss = criterion(outputs, load)
        test_losses.append(loss.item())
        
avg_test_loss = np.mean(test_losses)
print(f'验证集的平均损失: {avg_test_loss:.6f}')

Epoch [1/300], Train Loss: 0.985093, Val Loss: 0.935796
Epoch [2/300], Train Loss: 0.905992, Val Loss: 0.840978
Epoch [3/300], Train Loss: 0.810242, Val Loss: 0.769283
Epoch [4/300], Train Loss: 0.783066, Val Loss: 0.769197
Epoch [5/300], Train Loss: 0.732946, Val Loss: 0.718743
Epoch [6/300], Train Loss: 0.712769, Val Loss: 0.780821
Epoch [7/300], Train Loss: 0.733789, Val Loss: 0.708863
Epoch [8/300], Train Loss: 0.688339, Val Loss: 0.721906
Epoch [9/300], Train Loss: 0.661975, Val Loss: 0.662260
Epoch [10/300], Train Loss: 0.643572, Val Loss: 0.618004
Epoch [11/300], Train Loss: 0.606397, Val Loss: 0.642458
Epoch [12/300], Train Loss: 0.623073, Val Loss: 0.600979
Epoch [13/300], Train Loss: 0.594586, Val Loss: 0.624365
Epoch [14/300], Train Loss: 0.577726, Val Loss: 0.546674
Epoch [15/300], Train Loss: 0.540072, Val Loss: 0.568092
Epoch [16/300], Train Loss: 0.544303, Val Loss: 0.522844
Epoch [17/300], Train Loss: 0.549366, Val Loss: 0.583708
Epoch [18/300], Train Loss: 0.515229, Va

In [7]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, 'best_lstm_model.pth')
print(f'保存最佳模型，验证损失: {best_val_loss:.6f}')

保存最佳模型，验证损失: 0.091581


In [5]:
# 在适当的位置调用，比如每个训练周期结束后
torch.cuda.empty_cache()
